# Homework 7
## Due Date:  Wednesday, October 25th at 11:59 PM

# Problem 1:  Linked List Class
Write a linked list class called `LinkedList`.  Remember, a singly linked list is made up of nodes each of which contain a value and a pointer.  The first node is called the "head node".

Here are the required methods:
* `__init__(self, head)` where `head` is the value of the head node.  You could make the head node an attribute.
* `__len__(self)`: Returns the number of elements in the linked list.
* `__getitem__(self, index)` returns the value of the node corresponding to `index`.  Include checks to make sure that `index` is not out of range and that the user is not trying to index and empty list.
* `__repr__(self)` returns `LinkedList(head_node)`.
* `insert_front(self, element)` inserts a new node with value `element` at the beginning of the list.
* `insert_back(self, element)` inserts a new node with value `element` at the end of the list.

In [1]:
class Node:
    def __init__(self, element, next_node=None):
        self.value = element
        self.next_node = next_node
        
    def update_next_node(self, node):
        self.next_node = node

class LinkedList:
    def __init__(self, head):
        self.head_node = Node(head)
        self.tail_node = self.head_node
        self.size = 1
        
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        if isinstance(index, slice):
            raise TypeError('Slice is not supported.')
        if 0 == self.size:
            raise IndexError('The list is empty.')
        if index != int(index):
            raise TypeError('Invalid index.')
        if index < 0 or index >= self.size:
            raise IndexError('Out of range.')
        curr_node = self.head_node
        for _ in range(index):
            curr_node = curr_node.next_node
        return curr_node.value
    
    def __repr__(self):
        return 'LinkedList({})'.format(self.head_node)
    
    def insert_front(self, element):
        self.head_node = Node(element, self.head_node)
        self.size += 1
        
    def insert_back(self, element):
        self.tail_node.update_next_node(Node(element))
        self.tail_node = self.tail_node.next_node
        self.size += 1

In [2]:
# Example
a = LinkedList(1)
a.insert_front(10)
a.insert_back(100)
a.insert_back(1000)
a.insert_front(-10)
a.insert_back(5)
print(len(a))
print(repr(a))
print(', '.join([str(a[i]) for i in range(len(a))]))

6
LinkedList(<__main__.Node object at 0x111803ef0>)
-10, 10, 1, 100, 1000, 5


# Problem 2:  Binary Tree Class
A binary search tree is a binary tree with the invariant that for any particular node the left child is smaller and the right child is larger. Create the class `BinaryTree` with the following specifications:

`__init__(self)`: Constructor takes no additional arguments

`insert(self, val)`: This method will insert `val` into the tree

(Optional) `remove(self, val)`: This will remove `val` from the tree.
1. If the node to be deleted has no children then just remove it.
2. If the node to be deleted has only one child, remove the node and replace it with its child.
3. If the node to be deleted has two children, replace the node to be deleted with the maximum value in the left subtree.  Finally, delete the node with the maximum value in the left-subtree.

`getValues(self. depth)`: Return a list of the entire row of nodes at the specified depth with `None` at the index if there is no value in the tree. The length of the list should therefore be $2^{\text{depth}}$. 

Here is a sample output:

```python
bt = BinaryTree()
arr = [20, 10, 17, 14, 3, 0]
for i in arr:
    bt.insert(i)

print("Height of binary tree is {}.\n".format(len(bt)))
for i in range(len(bt)):
    print("Level {0} values: {1}".format(i, bt.getValues(i)))
```

```
Height of binary tree is 4.

Level 0 values: [20]
Level 1 values: [10, None]
Level 2 values: [3, 17, None, None]
Level 3 values: [0, None, 14, None, None, None, None, None]
```

Note that you do not need to format your output in this way.  Nor are you required to implement a `__len__` method to compute the height of the tree.  I did this because it was convenient for illustration purposes.  This example is simply meant to show you some output at each level of the tree.

In [1]:
class Node:
    def __init__(self, value, depth=0, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.parent = None
        self.depth = depth
        
    def set_left(self, left):
        self.left = left
        if left is not None:
            left.parent = self
            left.set_depth(self.depth + 1)
        
    def set_right(self, right):
        self.right = right
        if right is not None:
            right.parent = self
            right.set_depth(self.depth + 1)
        
    def set_parent(self, parent):
        self.parent = parent
        
    def set_depth(self, depth):
        self.depth = depth
        
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value
    
    def __le__(self, other):
        return self.value <= other.value
    
    def __ne__(self, other):
        return self.value != other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __ge__(self, other):
        return self.value >= other.value

class BinaryTree:
    def __init__(self):
        self.root = None
    
    def insert(self, val):
        new_node = Node(val)
        if self.root is None:
            self.root = new_node
            return
        curr = self.root
        while True:
            if curr < new_node:
                if curr.right is None:
                    curr.set_right(new_node)
                    return
                else:
                    curr = curr.right
                    continue
            elif new_node < curr:
                if curr.left is None:
                    curr.set_left(new_node)
                    return
                else:
                    curr = curr.left
                    continue
            elif new_node == curr:
                new_node.set_left(curr.left)
                curr.set_left(new_node)
                if new_node.left is not None and curr.parent is not None and curr.parent < curr:
                    new_node.set_right(new_node.left.right)
                    new_node.left.set_right(None)
                # update depth for the subtree
                self.getInOrder(new_node)
                return
    
    def remove(self, val):
        curr = self.root
        while curr is not None:
            if val < curr.value:
                curr = curr.left
                continue
            elif curr.value < val:
                curr = curr.right
                continue
            else:
                del_start = curr
                while True:
                    if curr.left is None or curr.left != curr:
                        del_end = curr
                        break
                    else:
                        curr = curr.left
                del_start.set_left(del_end.left)
                
                del_left = del_start.left
                del_right = del_start.right
                del_parent = del_start.parent
                if del_parent is not None:
                    on_right = del_parent < del_start
                del_start_parent = del_parent
                while True:
                    if del_left is not None:
                        new_del_left = del_left.left
                        new_del_right = del_left.right
                        if del_parent is not None:
                            if on_right:
                                del_parent.set_right(del_left)
                            else:
                                del_parent.set_left(del_left)
                        else:
                            self.root = del_left
                            self.root.set_depth(0)
                            self.root.set_parent(None)
                            del_start_parent = self.root
                        del_left.set_right(del_right)
                        del_parent = del_left
                        del_left = new_del_left
                        del_right = new_del_right
                        on_right = False
                    elif del_right is not None:
                        if on_right:
                            del_parent.set_right(del_right)
                        else:
                            del_parent.set_left(del_right)
                        break
                    else:
                        if on_right:
                            del_parent.set_right(None)
                        else:
                            del_parent.set_left(None)
                        break
                # update depth for the subtree
                self.getInOrder(del_start_parent)
                break
    
    def getValues(self, depth):
        inorder = self.getInOrder(self.root)
        values = []
        for node in inorder:
            if node.depth > depth:
                continue
            elif node.depth == depth:
                values.append(node.value)
            else:
                if node.left is None:
                    values += [None] * 2**(depth-node.depth-1)
                if node.right is None:
                    values += [None] * 2**(depth-node.depth-1)
        return values
    
    def getInOrder(self, node):
        # in-order traversal
        # update depth
        inorder = []
        def _inorder(node, inorder):
            if node is None:
                return
            elif node.parent is not None:
                node.set_depth(node.parent.depth + 1)
            _inorder(node.left, inorder)
            inorder.append(node)
            _inorder(node.right, inorder)
        _inorder(node, inorder)
        return inorder
    
    def maxDepth(self):
        inorder = self.getInOrder(self.root)
        return max([i.depth for i in inorder])
    
    def __len__(self):
        return self.maxDepth() + 1

In [2]:
# example 1
bt = BinaryTree()
arr = [20, 10, 17, 14, 3, 0]
for i in arr:
    bt.insert(i)

print("Height of binary tree is {}.\n".format(len(bt)))
for i in range(len(bt)):
    print("Level {0} values: {1}".format(i, bt.getValues(i)))

Height of binary tree is 4.

Level 0 values: [20]
Level 1 values: [10, None]
Level 2 values: [3, 17, None, None]
Level 3 values: [0, None, 14, None, None, None, None, None]


In [3]:
# example 2
bt = BinaryTree()
arr = [1, 2, -5, 2, 2, 3, -6, -3]
for i in arr:
    bt.insert(i)

print("Height of binary tree is {}.\n".format(len(bt)))
for i in range(len(bt)):
    print("Level {0} values: {1}".format(i, bt.getValues(i)))
print()

arr = [10, 1, 2]
for i in arr:
    bt.remove(i)
print('Try removing {}.'.format(', '.join([str(i) for i in arr])))

print("Height of binary tree is {}.\n".format(len(bt)))
for i in range(len(bt)):
    print("Level {0} values: {1}".format(i, bt.getValues(i)))

Height of binary tree is 4.

Level 0 values: [1]
Level 1 values: [-5, 2]
Level 2 values: [-6, -3, 2, 3]
Level 3 values: [None, None, None, None, 2, None, None, None]

Try removing 10, 1, 2.
Height of binary tree is 3.

Level 0 values: [-5]
Level 1 values: [-6, 3]
Level 2 values: [None, -3, None, None]


# Problem 3:  Peer Evaluations
Evaluate the members of your group for Milestone 1.  Please follow the instructions in the provided survey.  The survey can be found here:  [Milestone 1 Peer Evaluation](https://harvard.az1.qualtrics.com/jfe/form/SV_0JnuXbE5QjLCrKB).

# Problem 4:  Course Evaluation
Please take the [Course Evaluation](https://docs.google.com/forms/d/e/1FAIpQLSdDyrtf_aByU4xNeLMSmDrFCJ2OLDrK1Q7ZoeTd2Whf_cdRrw/viewform?usp=sf_link).